#💾 **Importando JSON.**

##📌 Bibliotecas.

In [293]:
import pandas as pd
import numpy as np
import re

In [294]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [295]:
telecomX = pd.read_json('/content/drive/MyDrive/Colab Notebooks/CHALLENGE: TELECOM X./TelecomX_Data.json')

#🧰 **Transformación.**

##🛠️ **Normalización de Columnas.**

In [296]:
##Creo función para normalizar columnas: 'customer','phone','internet','account'
#Lista con nombre de columnas a normalizar.
lista_columnas_normalizar = ['customer','phone','internet','account']
#Contiene columnas normalizadas.
columnas_normalizadas = {}
#normalizar_columnas normaliza columnas.
def normalizar_columnas(columnas):
  global columnas_normalizadas
  for columnas in lista_columnas_normalizar:
    normalizado = pd.json_normalize(telecomX[columnas])
    columnas_normalizadas[columnas] = normalizado
  return

#Normalizar columnas.
normalizar_columnas(lista_columnas_normalizar)
#Unimos información normalizada en df_telecomX_datos.
df_telecomX_datos = pd.concat([telecomX['customerID'],telecomX['Churn'],columnas_normalizadas['customer'],
                               columnas_normalizadas['phone'],columnas_normalizadas['internet'],
                               columnas_normalizadas['account']],axis=1)
df_telecomX_datos.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


##🛠️**Identificación de Inconsistencias en Datos.**

In [297]:
#Verificación de datos nulos, duplicados, error de formatos, inconsistencias en las categorías.
#Lista de columnas.
lista_columnas = df_telecomX_datos.columns
#Contiene la información del tipo de registro en cada columna en df_telecomX_datos
tipo_dato_columna_telecomx = {}

#Función para verificar tipo de registro en las columnas.
def verificar_datos(columnas):
  global tipo_dato_columna_telecomx
  for c in lista_columnas:
    tipo_registro = df_telecomX_datos[c].unique()
    tipo_dato_columna_telecomx[c] = tipo_registro
  return

#Verificar datos.
verificar_datos(lista_columnas)

tipo_dato_columna_telecomx['PaymentMethod']

array(['Mailed check', 'Electronic check', 'Credit card (automatic)',
       'Bank transfer (automatic)'], dtype=object)

In [298]:
tipo_dato_columna_telecomx.keys()

dict_keys(['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Charges.Monthly', 'Charges.Total'])

In [299]:
#1. Columna 'SeniorCitizen'(0,1)(objet)Se puede convertir sus valores a 'Yes' o 'No'
#2. Columnas 'Contract y 'PaymentMethod' podríamos unir palabras por un guion (-) para tokenizar
#3. Columna 'PaymentMethod' remover (), reemplazar espacios por guion.
#4. Columna 'Charges.Total' convertir a tipo float64.
#5. Tratamiento preventivo para columnas tipo texto : Remover espacios al inicio y final,convertimos a minúscula, removemos valores nulos por desconocido.
#6. Tratamiento preventivo para columnas con caracteres numericos y conversión tipo float64.

##🛠️**Manejo de Inconsistencias en Datos.**

In [300]:
#Tratamiento mínimo para columnas tipo cadena de texto.
#Tratamiento de datos en columna 'SeniorCitizen'(0,1)(objet)Se puede convertir sus valores a 'Yes' o 'No'
df_telecomX_datos['SeniorCitizen'] = df_telecomX_datos['SeniorCitizen'].astype('str').replace('0','No').replace('1','Yes')


#Remover espacios al inicio y final,convertimos a minúscula, removemos valores nulos por desconocido.
columnas_texto = ['Churn','gender','SeniorCitizen','Partner','Dependents','PhoneService','MultipleLines',
                  'InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport',
                  'StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod','Charges.Total']
for col in columnas_texto:
    df_telecomX_datos[col] = df_telecomX_datos[col].astype(str).str.strip().str.lower().replace('', np.nan).fillna('desconocido')


In [301]:
#Columnas 'Contract y 'PaymentMethod' podríamos unir palabras por un guion (-) para tokenizar.
#Son las columnas cuyos registros deseamos unir con un guion.
columnas_espacios = ['Contract','PaymentMethod','InternetService']
for col in columnas_espacios:
  df_telecomX_datos[col] = df_telecomX_datos[col].str.replace(' ','-')

In [302]:
#Columna 'PaymentMethod' remover (), reemplazar espacios por guion.
df_telecomX_datos['PaymentMethod'] = df_telecomX_datos['PaymentMethod'] \
    .str.replace('(', '-', regex=False) \
    .str.replace(')', '', regex=False) \
    .str.replace(' ', '-', regex=False)

In [303]:
#Columna 'Charges.Monthly','Charges.Total' Limpieza preventiva de caracteres menos numericos y el punto.
columnas_numericas = ['Charges.Monthly','Charges.Total']
for col in columnas_numericas:
  df_telecomX_datos[col] = pd.to_numeric(df_telecomX_datos[col].astype(str).str.replace(r'[^0-9.]', '', regex=True), errors='coerce')

In [304]:
df_telecomX_datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   object 
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


In [305]:
df_telecomX_datos_copy = df_telecomX_datos.copy()
telecomX_vistas_1_10 = df_telecomX_datos_copy[['customerID','Churn','gender','SeniorCitizen','Partner','Dependents',
                                               'tenure','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup']]
telecomX_vistas_1_10.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup
0,0002-ORFBO,no,female,no,yes,yes,9,yes,no,dsl,no,yes
1,0003-MKNFE,no,male,no,no,no,9,yes,yes,dsl,no,no
2,0004-TLHLJ,yes,male,no,no,no,4,yes,no,fiber-optic,no,no
3,0011-IGKFF,yes,male,yes,yes,no,13,yes,no,fiber-optic,no,yes
4,0013-EXCHZ,yes,female,yes,yes,no,3,yes,no,fiber-optic,no,no


In [306]:
df_telecomX_datos_copy = df_telecomX_datos.copy()
telecomX_vistas_11_21 = df_telecomX_datos_copy[['customerID','Churn', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
                                               'Contract','PaperlessBilling', 'PaymentMethod', 'Charges.Monthly', 'Charges.Total']]
telecomX_vistas_11_21.head()

,customerID,Churn,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,no,no,yes,yes,no,one-year,yes,mailed-check,65.6,593.30
1,0003-MKNFE,no,no,no,no,yes,month-to-month,no,mailed-check,59.9,542.40
2,0004-TLHLJ,yes,yes,no,no,no,month-to-month,yes,electronic-check,73.9,280.85
3,0011-IGKFF,yes,yes,no,yes,yes,month-to-month,yes,electronic-check,98.0,1237.85
4,0013-EXCHZ,yes,no,yes,yes,no,month-to-month,yes,mailed-check,83.9,267.40


#📊 Carga y análisis

In [307]:
df_telecomX_datos_copy.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,no,female,no,yes,yes,9,yes,no,dsl,...,yes,no,yes,yes,no,one-year,yes,mailed-check,65.6,593.30
1,0003-MKNFE,no,male,no,no,no,9,yes,yes,dsl,...,no,no,no,no,yes,month-to-month,no,mailed-check,59.9,542.40
2,0004-TLHLJ,yes,male,no,no,no,4,yes,no,fiber-optic,...,no,yes,no,no,no,month-to-month,yes,electronic-check,73.9,280.85
3,0011-IGKFF,yes,male,yes,yes,no,13,yes,no,fiber-optic,...,yes,yes,no,yes,yes,month-to-month,yes,electronic-check,98.0,1237.85
4,0013-EXCHZ,yes,female,yes,yes,no,3,yes,no,fiber-optic,...,no,no,yes,yes,no,month-to-month,yes,mailed-check,83.9,267.40


#📄Informe final